# Configuracion

Estos primeros bloques son de importar librerías, y configurar las credenciales de la API para poder descargar las imágenes.

In [ ]:
# Importar librerías
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image  
import PIL
from math import cos, radians
from sentinelhub import (SHConfig, DataCollection, SentinelHubCatalog, SentinelHubRequest, BBox, bbox_to_dimensions,
                         CRS, MimeType, Geometry)
import shutil

In [2]:
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

## Credenciales

Este bloque es para asignar las credenciales de la cuenta de Copernicus (https://dataspace.copernicus.eu/)
Se debe ingresar a la Dashboard (https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings) y crear un nuevo OAth Client, desde ahí, copiar los 'Client ID' y 'Client Secret'.

In [3]:
config = SHConfig()

config.sh_client_id = ""
config.sh_client_secret = ""
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

config = SHConfig("cdse")

# Area

Estos bloques son para asignar el área geográfica para descargar. Son Bounding Boxs rectangulares con las coordenadas WGS84

El formato es: Coordenadas Esquina Inferior Izquierda, Coordenadas Esquina Superior Derecha

Se elige también la resolución en metros/pixel y se muestra el tamaño (resolución en pixeles) aproximado de la imagen a descargar. 

> LA API NO ADMITE TAMAÑOS DE IMÁGENES MAYORES A 2500 PIXELES DE ALTO O ANCHO

Las coordenadas están hechas de tal forma que ninguna imagen queda mayor a 2500 pixles por ningún lado.

In [4]:
# Specify the path to your Excel file and the name of the sheet
excel_file = 'dbCoordenadas - CompletaCropeadasBonitas.xlsx'
sheet_name = 'Sheet1'

# Read the specific column from the Excel file into a DataFrame
column_name = 'B'  # Replace 'A' with the column letter or name you want to read
df = pd.read_excel(excel_file, sheet_name=sheet_name, usecols=column_name)

#df['Bbox'] = df['Bbox'].str.slice(start=7)
#df['Bbox'] = df['Bbox'].str[:-1]

# Split the string numbers into multiple columns
df[['longMin', 'latMin', 'longMax', 'latMax']] = df['Bbox'].str.split(',', expand=True)

# Convert the new columns to numeric
df[['longMin', 'latMin', 'longMax', 'latMax']] = df[['longMin', 'latMin', 'longMax', 'latMax']].apply(pd.to_numeric)

# Drop the original column
df.drop(columns=['Bbox'], inplace=True)


listaDeListas = df.values.tolist()
listaDeListas

[[-68.769951, -21.022502, -68.581467, -20.926971],
 [-68.768406, -21.020739, -68.660259, -20.929697],
 [-68.8484, -21.032276, -68.767548, -20.97795],
 [-69.317894, -20.095433, -69.223137, -20.018355],
 [-68.864365, -21.941931, -68.782654, -21.896544],
 [-68.73991, -22.033616, -68.642578, -21.958012],
 [-68.929682, -22.411187, -68.865566, -22.356506],
 [-68.842907, -22.268923, -68.949594, -22.358808],
 [-68.819475, -22.158394, -68.921013, -22.268923],
 [-69.678211, -22.348806, -69.614182, -22.305614],
 [-69.642334, -22.419598, -69.591179, -22.367698],
 [-69.704647, -22.646017, -69.638729, -22.580571],
 [-69.904118, -22.661383, -69.81966, -22.55283],
 [-70.175986, -22.611713, -70.157747, -22.598837],
 [-70.123329, -22.613615, -70.093932, -22.59119],
 [-70.211048, -22.730907, -70.144787, -22.666769],
 [-69.337463, -22.843749, -69.213181, -22.72869],
 [-69.432907, -22.926144, -69.279785, -22.790742],
 [-69.826355, -22.878903, -69.808974, -22.863561],
 [-69.005814, -23.026028, -69.151382, -

In [5]:
dfRegion = pd.read_excel(excel_file, sheet_name='Sheet1', usecols='C')

listaDeListasRegion = dfRegion.values.tolist()

In [6]:
listaRegiones = []
for regiones in listaDeListasRegion:
    listaRegiones.append(regiones[0])
    print(regiones[0])

1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
4
5
5
5
5
13
13
13
6
6


In [7]:
excelCoordenadas = 'coordenadasAtlas.xlsx'
dff = pd.read_excel(excelCoordenadas)
dff = dff[['latitud', 'longitud','region']]

coooooordenadas = dff[['latitud','longitud']].values.tolist()
image_size_pixels = 2300
resolution_meters_per_pixel = 10

In [8]:
listaRegionesAtlas = dff['region'].tolist()


# De aquí para abajo poner el chamuyo de los BBOX

Al final, la variable `coooooooordenadas` debería ser igual a `puntosAUsar`. Con eso debería andar bien todo lo demás.

In [9]:
cantidadDeCoordenadas = len(coooooordenadas)
print(cantidadDeCoordenadas)

puntosContenidos = [[] * cantidadDeCoordenadas for _ in range(cantidadDeCoordenadas)]  # Inicializar una matriz de ceros
print(len(puntosContenidos))

nPuntosContenidos = [[],[],[]]

1657
1657


In [10]:
import math

# Función para calcular la distancia haversine entre dos puntos
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en kilómetros
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

# Función para calcular el BBOX a partir de un punto y una distancia
def calculate_bbox(lat, lon, distance):
    # Convertir la distancia de kilómetros a grados de latitud/longitud
    lat_offset = distance / 111.0
    lon_offset = distance / (111.0 * math.cos(math.radians(lat)))

    # Calcular las coordenadas del BBOX
    lat_min = lat - lat_offset
    lat_max = lat + lat_offset
    lon_min = lon - lon_offset
    lon_max = lon + lon_offset

    return lat_min, lon_min, lat_max, lon_max

# Función para verificar si un punto está dentro del BBOX
def is_point_inside_bbox(point, bbox):
    lat, lon = point
    lat_min, lon_min, lat_max, lon_max = bbox
    return lat_min <= lat <= lat_max and lon_min <= lon <= lon_max

# Lista de puntos de coordenadas (latitud, longitud)
puntos = coooooordenadas

# Verificar si cada punto está contenido en el BBOX de cada otro punto

for i, punto in enumerate(puntos):
    indicessss = 0
    #print(f"Punto {i + 1}: {punto}")
    for j, otro_punto in enumerate(puntos):
        if i != j:  # Evitar verificar el punto consigo mismo
            distancia = 11  # Distancia en kilómetros
            lat_min, lon_min, lat_max, lon_max = calculate_bbox(otro_punto[0], otro_punto[1], distancia)
            bbox = (lat_min, lon_min, lat_max, lon_max)
            bboxChueco = (lon_min, lat_min, lon_max, lat_max)
            if is_point_inside_bbox(punto, bbox):
                #print(f"   Está dentro del BBOX de Punto {j + 1}, {otro_punto}")
                puntosContenidos[i].append(otro_punto)
                indicessss += 1
    nPuntosContenidos[0].append(indicessss)
    nPuntosContenidos[1].append(punto)


In [11]:
dfPuntosConetnidos = pd.DataFrame(nPuntosContenidos)
dfPuntosConetnidos = dfPuntosConetnidos.transpose()
#dfPuntosConetnidos

In [12]:
puntosListos = []
puntosAUsar = []

# Sort the DataFrame by the column in descending order
sorted_df = dfPuntosConetnidos.sort_values(by=0, ascending=False)

# Print the values of the column in descending order

for cantidadContenidos, CoordenadaCentral in zip(sorted_df[0], sorted_df[1]):
    #print(cantidadContenidos, CoordenadaCentral)
    if CoordenadaCentral not in puntosListos:
        print(f' {CoordenadaCentral} No estaba Lista')
        indice = puntos.index(CoordenadaCentral)
        puntosAUsar.append(CoordenadaCentral)
        #print(indice)
        #print(puntosContenidos[indice])
        for naranja in puntosContenidos[indice]:
            puntosListos.append(naranja)
    else:
        print(f'{CoordenadaCentral} SI Estaba Lista')



 [-27.445722, -70.322525] No estaba Lista
[-27.44499, -70.310233] SI Estaba Lista
[-27.46153, -70.273949] SI Estaba Lista
[-27.4753, -70.293002] SI Estaba Lista
[-27.479572, -70.290856] SI Estaba Lista
[-27.425344, -70.310689] SI Estaba Lista
[-27.424436, -70.314138] SI Estaba Lista
[-27.460348, -70.271912] SI Estaba Lista
[-27.479491, -70.282951] SI Estaba Lista
[-27.479125, -70.283473] SI Estaba Lista
[-27.47506, -70.276949] SI Estaba Lista
[-27.480313, -70.275997] SI Estaba Lista
[-27.421253, -70.313716] SI Estaba Lista
[-27.482247, -70.27772] SI Estaba Lista
[-27.453323, -70.257149] SI Estaba Lista
[-27.451355, -70.257268] SI Estaba Lista
[-27.452697, -70.256525] SI Estaba Lista
[-27.453066, -70.256711] SI Estaba Lista
[-27.434914, -70.268554] SI Estaba Lista
[-27.491802, -70.296718] SI Estaba Lista
[-27.417206, -70.323642] SI Estaba Lista
[-27.454988, -70.2548] SI Estaba Lista
[-27.41702, -70.323235] SI Estaba Lista
[-27.492738, -70.296091] SI Estaba Lista
[-27.414968, -70.313085]

In [13]:

# Assuming df is your DataFrame and 'value' is the value you want to find the index of

for latitud,longitud in puntosAUsar:
    
    indexLati = dff.index[dff['latitud'] == latitud].tolist()

    if indexLati:
        print("Index of latitud", latitud, ":", indexLati)        
        listaRegiones.append(dff.loc[indexLati[0], 'region'])
    else:
        print(latitud, "not found in the DataFrame")
    
    
    
    
    indexLongi = dff.index[dff['longitud'] == longitud].tolist()
    
    if indexLongi:
        print("Index of longitud", longitud, ":", indexLongi)
    else:
        print(longitud, "not found in the DataFrame")



Index of latitud -27.445722 : [506]
Index of longitud -70.322525 : [506]
Index of latitud -32.414075 : [1322]
Index of longitud -71.012311 : [1322]
Index of latitud -32.254211 : [1397]
Index of longitud -70.864438 : [1397]
Index of latitud -27.560542 : [531]
Index of longitud -70.334624 : [531]
Index of latitud -32.407146 : [1222]
Index of longitud -70.867237 : [1222]
Index of latitud -32.573543 : [1275, 1286, 1287]
Index of longitud -70.853529 : [1275, 1286, 1287]
Index of latitud -33.084802 : [1590]
Index of longitud -70.893708 : [1590]
Index of latitud -27.546234 : [534]
Index of longitud -70.187518 : [534]
Index of latitud -32.266911 : [1339]
Index of longitud -71.017536 : [1339]
Index of latitud -30.268999 : [938]
Index of longitud -71.182789 : [938]
Index of latitud -34.027111 : [1623]
Index of longitud -70.914022 : [1623]
Index of latitud -32.153728 : [1400]
Index of longitud -70.874667 : [1400]
Index of latitud -31.411625 : [1066]
Index of longitud -71.056017 : [1066]
Index of 

In [14]:
valorTOmate = dff.loc[506, 'region']

print(valorTOmate)

3


In [15]:
coooooordenadas = puntosAUsar

In [16]:
for lat, lon in coooooordenadas:
    #print(lat, lon)

    #latitude = -23.417694
    #longitude = -70.038786

    # Convert resolution from meters to degrees for both latitude and longitude
    resolution_degrees_lat = resolution_meters_per_pixel / 111132.92  # Approximate 1 degree latitude distance in meters
    resolution_degrees_lon = resolution_meters_per_pixel / (
            111132.92 * cos(radians(lat)))  # Approximate 1 degree longitude distance in meters

    # Compute the extent of the image in degrees
    image_extent_lat = resolution_degrees_lat * image_size_pixels / 2
    image_extent_lon = resolution_degrees_lon * image_size_pixels / 2

    # Compute bounding box coordinates
    min_lat = lat - image_extent_lat
    max_lat = lat + image_extent_lat
    min_lon = lon - image_extent_lon
    max_lon = lon + image_extent_lon

    # Bounding box coordinates in WGS84 degrees
    #bounding_box = (min_lat, min_lon, max_lat, max_lon)
    #bounding_box = [min_lat, min_lon, max_lat, max_lon]
    bounding_box = [min_lon, min_lat, max_lon, max_lat]
    listaDeListas.append(bounding_box)

    resolution = 10
    aoi_bbox = BBox(bbox=bounding_box, crs=CRS.WGS84)
    aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

    print(aoi_size)

    if aoi_size[0] > 2500 or aoi_size[1] > 2500:
        #print(f'Image shape at {lat},{lon} m resolution: {aoi_size} pixels')
        #min_lat = min_lat*0.55
        #max_lat = max_lat*0.55
        #min_lon = min_lon*0.7
        #max_lon = max_lon*0.7
        bounding_box = [min_lat, min_lon, max_lat, max_lon]
        bounding_box = [min_lon, min_lat, max_lon, max_lat]
        print(bounding_box)
        listaDeListas.append(bounding_box)
        aoi_bbox = BBox(bbox=bounding_box, crs=CRS.WGS84)
        aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)
        print(aoi_size)

        if aoi_size[0] > 2500 or aoi_size[1] > 2500:
            print(f'Image shape at {lat},{lon} m resolution: {aoi_size} pixels')


(2281, 2317)
(2263, 2338)
(2266, 2335)
(2280, 2318)
(2266, 2335)
(2266, 2335)
(2264, 2337)
(2283, 2315)
(2263, 2338)
(2262, 2338)
(2263, 2338)
(2266, 2335)
(2263, 2338)
(2263, 2338)
(2284, 2313)
(2261, 2340)
(2260, 2341)
(2287, 2309)
(2282, 2315)
(2263, 2337)
(2265, 2337)
(2283, 2314)
(2278, 2320)
(2265, 2336)
(2280, 2317)
(2264, 2336)
(2266, 2334)
(2264, 2336)
(2262, 2339)
(2268, 2333)
(2286, 2311)
(2276, 2324)
(2269, 2329)
(2260, 2341)
(2272, 2326)
(2273, 2326)
(2306, 2289)
(2281, 2316)
(2283, 2314)
(2259, 2342)
(2261, 2340)
(2283, 2314)
(2262, 2338)
(2274, 2325)
(2268, 2331)
(2286, 2313)
(2299, 2297)
(2262, 2339)
(2263, 2337)
(2285, 2312)
(2263, 2337)
(2287, 2310)
(2265, 2335)
(2260, 2340)
(2268, 2332)
(2287, 2310)
(2265, 2335)
(2275, 2323)
(2264, 2337)
(2265, 2336)
(2272, 2327)
(2264, 2336)
(2275, 2323)
(2269, 2332)
(2273, 2328)
(2263, 2337)
(2283, 2315)
(2285, 2312)
(2260, 2340)
(2263, 2336)
(2258, 2343)
(2283, 2313)
(2280, 2317)
(2277, 2321)
(2269, 2331)
(2307, 2288)
(2287, 2309)

In [17]:
listaDeListas = [[round(num, 5) for num in sublist] for sublist in listaDeListas]

for coordenadas in listaDeListas:

    resolution = 10
    aoi_bbox = BBox(bbox=coordenadas, crs=CRS.WGS84)
    aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)
    
    print(f'Image shape at {coordenadas} m resolution: {aoi_size} pixels')
    if aoi_size[0] > 2500 or aoi_size[1] > 2500:
        print(f'Image shape at {coordenadas} m resolution: {aoi_size} pixels')

Image shape at [-68.76995, -21.0225, -68.58147, -20.92697] m resolution: (1961, 1053) pixels
Image shape at [-68.76841, -21.02074, -68.66026, -20.9297] m resolution: (1126, 1006) pixels
Image shape at [-68.8484, -21.03228, -68.76755, -20.97795] m resolution: (841, 600) pixels
Image shape at [-69.31789, -20.09543, -69.22314, -20.01835] m resolution: (989, 855) pixels
Image shape at [-68.86437, -21.94193, -68.78265, -21.89654] m resolution: (845, 501) pixels
Image shape at [-68.73991, -22.03362, -68.64258, -21.95801] m resolution: (1006, 835) pixels
Image shape at [-68.92968, -22.41119, -68.86557, -22.35651] m resolution: (660, 605) pixels
Image shape at [-68.84291, -22.26892, -68.94959, -22.35881] m resolution: (1099, 994) pixels
Image shape at [-68.81947, -22.15839, -68.92101, -22.26892] m resolution: (1048, 1223) pixels
Image shape at [-69.67821, -22.34881, -69.61418, -22.30561] m resolution: (657, 481) pixels
Image shape at [-69.64233, -22.4196, -69.59118, -22.3677] m resolution: (52

# Scripts

Estos scripts se usan para descirle a la API lo que tiene que descargar.

En este caso, son Todas las bandas del satelite.

In [18]:
# Script Sólo RGB

evalscriptTrueColor = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: [
          "B02",
          "B03",
          "B04",
        ],
        units: "DN",
      },
    ],
    output: {
      id: "default",
      bands: 3,
      sampleType: SampleType.UINT16,
    },
  }
}

function evaluatePixel(sample) {
  return [
    sample.B02,
    sample.B03,
    sample.B04,
  ]
}
"""

# Rango de Tiempo
Este bloque determina el rango de tiempo a descargar. Originalmente está en 2017, 4, 1 - 2024, 4,3
Se deben definir cuántas imágenes descargar en dicho tiempo, por ejemplo 50 imágenes entre el 2019 - 2024

Hay buenas imágenes desde abril del 2017 más menos

Se estiman unas 200 imágenes por BBOX

## Otros Rangos de tiempo

Originalmente está desde 2017-04-01 hasta 2024-04-22, pero por ejemplo si se quiere descargar SÓLO la imagen más reciente, habría que cambiarle el `end` a la fecha de HOY, y el `start`, a 1 mes anterior al `end`. Además, se tiene que cambiar el `n_chunks = 1`, para que SÓLO descargue una (1) imagen.

In [19]:
start = datetime.datetime(2017, 4, 1)
end = datetime.datetime(2024, 4, 22)
n_chunks = 200
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks+1)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)

Monthly time windows:

('2017-04-01', '2017-04-13')
('2017-04-13', '2017-04-26')
('2017-04-26', '2017-05-09')
('2017-05-09', '2017-05-22')
('2017-05-22', '2017-06-04')
('2017-06-04', '2017-06-17')
('2017-06-17', '2017-06-30')
('2017-06-30', '2017-07-13')
('2017-07-13', '2017-07-26')
('2017-07-26', '2017-08-07')
('2017-08-07', '2017-08-20')
('2017-08-20', '2017-09-02')
('2017-09-02', '2017-09-15')
('2017-09-15', '2017-09-28')
('2017-09-28', '2017-10-11')
('2017-10-11', '2017-10-24')
('2017-10-24', '2017-11-06')
('2017-11-06', '2017-11-19')
('2017-11-19', '2017-12-01')
('2017-12-01', '2017-12-14')
('2017-12-14', '2017-12-27')
('2017-12-27', '2018-01-09')
('2018-01-09', '2018-01-22')
('2018-01-22', '2018-02-04')
('2018-02-04', '2018-02-17')
('2018-02-17', '2018-03-02')
('2018-03-02', '2018-03-15')
('2018-03-15', '2018-03-27')
('2018-03-27', '2018-04-09')
('2018-04-09', '2018-04-22')
('2018-04-22', '2018-05-05')
('2018-05-05', '2018-05-18')
('2018-05-18', '2018-05-31')
('2018-05-31', '2018

# Mapa Puntos del Altas
Aquí se muestra el mapa de Chile con los puntos del Atlas
Hay unos puntos que salen en Argentina (no sé si será error del Atlas o alguna otra cosa, pero da lo mismo) (en todo caso, las borré en la nueva versión del Excel, así que no deberían salir, a no ser que se use el Excel viejo del Atlas)

In [20]:
import folium

# Create a map centered around Chile
m = folium.Map(location=[-35, -70], zoom_start=4)


for index, row in dff.iterrows():
    folium.Marker(location=[row['latitud'], row['longitud']]).add_to(m)



# Display the map
m

# Descargar
El primer bloque es para procesamiento de imágenes.
El factor es principalmente para el brillo de la imagen. Estaba originalmente en 2 / 1e4, que es un buen factor para el espectro visible.

In [21]:
for coordennnnada in listaDeListas:
    if coordennnnada[0]-coordennnnada[2] <0 or coordennnnada[1]-coordennnnada[3]:
        print(f'La coordenada en {coordennnnada} está BIEN')
        ajlhksflkajsfljkas = 1
    else:
        print(f'La coordenada en {coordennnnada} está MALA')

La weá en [-68.76995, -21.0225, -68.58147, -20.92697] está BIEN
La weá en [-68.76841, -21.02074, -68.66026, -20.9297] está BIEN
La weá en [-68.8484, -21.03228, -68.76755, -20.97795] está BIEN
La weá en [-69.31789, -20.09543, -69.22314, -20.01835] está BIEN
La weá en [-68.86437, -21.94193, -68.78265, -21.89654] está BIEN
La weá en [-68.73991, -22.03362, -68.64258, -21.95801] está BIEN
La weá en [-68.92968, -22.41119, -68.86557, -22.35651] está BIEN
La weá en [-68.84291, -22.26892, -68.94959, -22.35881] está BIEN
La weá en [-68.81947, -22.15839, -68.92101, -22.26892] está BIEN
La weá en [-69.67821, -22.34881, -69.61418, -22.30561] está BIEN
La weá en [-69.64233, -22.4196, -69.59118, -22.3677] está BIEN
La weá en [-69.70465, -22.64602, -69.63873, -22.58057] está BIEN
La weá en [-69.90412, -22.66138, -69.81966, -22.55283] está BIEN
La weá en [-70.17599, -22.61171, -70.15775, -22.59884] está BIEN
La weá en [-70.12333, -22.61361, -70.09393, -22.59119] está BIEN
La weá en [-70.21105, -22.7309

In [22]:
def divide_list(lst,divi):
    # Calculate the size of each sublist
    sublist_size = len(lst) // divi
    # Calculate the remainder for distribution
    remainder = len(lst) % divi
    # Initialize the starting index for slicing
    start = 0
    # Initialize an empty list to store the divided sublists
    divided_lists = []

    # Loop through 5 times
    for i in range(divi):
        # Calculate the end index for slicing
        end = start + sublist_size + (1 if i < remainder else 0)
        # Slice the original list and append to the divided lists
        divided_lists.append(lst[start:end])
        # Update the starting index for the next iteration
        start = end

    return divided_lists

# Example usage:
sublistsRegion = divide_list(listaRegiones,5)
sublists = divide_list(listaDeListas,5)


In [23]:
factor = 2 / 1e4
clipRange= (0, 1)
nombreBandas = ['B01', 'B02','B03','B04','B05','B06','B07','B08','B8A','B09','B11','B12']


# División de Coordenadas a descargar

La variable `listaDeListas` contiene todas las coordenadas a descargar (303 en total), pero se crea una variable llamada `sublists` que divide todo en 5 cinco, con el fin de descargar las imágenes usando 5 computadores en paralelo.

En paralelo, se crea una variable llamada `sublistsRegion`, esto es sólo para ordenar las carpetas por región según la coordenada.

A ambas variables mencionandas anteriormente, se las dividen en 11. Esto se hace debido a que cada cuenta gratis permite descargar 6 coordenadas antes de tener que cambiar la cuenta por otra.

Para hacer esta división se deben cambiar dos líneas del bloque de abajo según corresponda.

Por ejemplo:
Computador 1 tiene que poner en el for del bloque de abajo

`subManzanaRegion = divide_list(sublistsRegion[0],11)`

`subManzana = divide_list(sublists[0],11)`



Entonces sería:

- Computador 1:
    - `subManzanaRegion = divide_list(sublistsRegion[0],11)`
    - `subManzana = divide_list(sublists[0],11)`
- Computador 2:
    - `subManzanaRegion = divide_list(sublistsRegion[1],11)`
    - `subManzana = divide_list(sublists[1],11)`
- Computador 3:
    - `subManzanaRegion = divide_list(sublistsRegion[2],11)`
    - `subManzana = divide_list(sublists[2],11)`
- Computador 4:
    - `subManzanaRegion = divide_list(sublistsRegion[3],11)`
    - `subManzana = divide_list(sublists[3],11)`
- Computador 5:
    - `subManzanaRegion = divide_list(sublistsRegion[4],11)`
    - `subManzana = divide_list(sublists[4],11)`

# Procedimiento al cambio de Cuentas

Una vez se descarguen o coordenadas, se deben cambiar las credenciales, y también el ciclo for de abajo de la siguiente forma:

- Cuenta 01:
    - `for coordenadas, region in zip(subManzana[0], subManzanaRegion[0]):`
- Cuenta 02:
    - `for coordenadas, region in zip(subManzana[1], subManzanaRegion[1]):`
- Cuenta 03:
    - `for coordenadas, region in zip(subManzana[2], subManzanaRegion[2]):`
- Cuenta 04:
    - `for coordenadas, region in zip(subManzana[3], subManzanaRegion[3]):`
- Cuenta 05:
    - `for coordenadas, region in zip(subManzana[4], subManzanaRegion[4]):`
- Cuenta 06:
    - `for coordenadas, region in zip(subManzana[5], subManzanaRegion[5]):`
- Cuenta 07:
    - `for coordenadas, region in zip(subManzana[6], subManzanaRegion[6]):`
- Cuenta 08:
    - `for coordenadas, region in zip(subManzana[7], subManzanaRegion[7]):`
- Cuenta 09:
    - `for coordenadas, region in zip(subManzana[8], subManzanaRegion[8]):`
- Cuenta 10:
    - `for coordenadas, region in zip(subManzana[9], subManzanaRegion[9]):`
- Cuenta 11:
    - `for coordenadas, region in zip(subManzana[10], subManzanaRegion[10]):`

In [23]:
subManzanaRegion = divide_list(sublistsRegion[0],11)
subManzana = divide_list(sublists[0],11)

# Sólo RGB
El Bloque de abajo descarga sólo la versión RGB de las imágenes

In [24]:
cebolla = 0



for coordenadas, region in zip(subManzana[10], subManzanaRegion[10]):

    print (f'Descargando Coordenada {coordenadas}')

    resolution = 10
    aoi_bbox = BBox(bbox=coordenadas, crs=CRS.WGS84)
    aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

    for slot in slots:
        request_all_bands = SentinelHubRequest(
            evalscript= evalscriptTrueColor,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L2A.define_from(
                        name="s2", service_url="https://sh.dataspace.copernicus.eu"
                    ),
                    time_interval=slot,
                    other_args={"dataFilter": {"mosaickingOrder": "leastCC"}}           )
            ],
            responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
            bbox=aoi_bbox,
            size=aoi_size,
            config=config,
        )
        all_bands_img = request_all_bands.get_data()

        # Carpetas

        #nombreCarpetas = f'Detecciones/{listaDeListasRegion[cebolla][0]}/{coordenadas}'
        nombreCarpetas = f'Detecciones/{region}/{coordenadas}'
        carpetaRGB = f'{nombreCarpetas}/True Color'

        if not os.path.exists(nombreCarpetas):
            os.makedirs(nombreCarpetas)


        #Bandas RAW sin procesar
        indice = 0

        if not (np.max(all_bands_img[0]) - np.min(all_bands_img[0])) == 0:

            #for bandas in nombreBandas:

            #    carpetaRawCebollas = f'{nombreCarpetas}/{bandas}'

            #    if not os.path.exists(carpetaRawCebollas):
            #        os.makedirs(carpetaRawCebollas)

            #    data_array = all_bands_img[0][:,:,indice]


            #    data_array_normalized = (data_array - np.min(data_array)) / (np.max(data_array) - np.min(data_array))
            #    data_array_scaled = (data_array_normalized * (2**16 - 1)).astype(np.uint16)
            #    bandaRawCebolla = Image.fromarray(data_array_scaled)
            #    bandaRawCebolla.save(f'{carpetaRawCebollas}/{coordenadas} - {slot} - {bandas}.tiff')

                #terremoto = Image.open(f'{nombreCarpetas}/{coordenadas} - {slot} - {bandas}.tiff')
                #terremoto.save(f'{nombreCarpetas}/{coordenadas} - {slot} - {bandas}.png')

                #bandaRaw = (np.clip(all_bands_img[0][:,:,indice]*factor,*clipRange))*255
                #bandaRaw = bandaRaw.astype(np.float32)
                #bandaRaw = Image.fromarray(bandaRaw)
                #bandaRaw.save(f'{nombreCarpetas}/{coordenadas} - {slot} - {bandas}.tiff')
            #    indice += 1

            # Bandas Procesadas

            if not os.path.exists(carpetaRGB):
                os.makedirs(carpetaRGB)

            trueColor = (np.clip(all_bands_img[0][:,:,[2,1,0]]*factor,*clipRange))*255
            trueColor = trueColor.astype(np.uint8)
            trueColor = Image.fromarray(trueColor)
            trueColor.save(f'{carpetaRGB}/{coordenadas} - {slot} - trueColor.png')
            print(f'Imagen de coordenada {coordenadas} y tiempo {slot} ha sido guardada')
            cebolla += 1

        else:
            print(f'No hay imagen en el periodo {slot}')
            indice += 1
            cebolla += 1


Descargando Coordenada [-70.30423, -27.64971, -70.07081, -27.44275]
Imagen de coordenada [-70.30423, -27.64971, -70.07081, -27.44275] y tiempo ('2017-04-01', '2017-04-13') ha sido guardada
Imagen de coordenada [-70.30423, -27.64971, -70.07081, -27.44275] y tiempo ('2017-04-13', '2017-04-26') ha sido guardada
Imagen de coordenada [-70.30423, -27.64971, -70.07081, -27.44275] y tiempo ('2017-04-26', '2017-05-09') ha sido guardada
Imagen de coordenada [-70.30423, -27.64971, -70.07081, -27.44275] y tiempo ('2017-05-09', '2017-05-22') ha sido guardada
Imagen de coordenada [-70.30423, -27.64971, -70.07081, -27.44275] y tiempo ('2017-05-22', '2017-06-04') ha sido guardada
Imagen de coordenada [-70.30423, -27.64971, -70.07081, -27.44275] y tiempo ('2017-06-04', '2017-06-17') ha sido guardada
Imagen de coordenada [-70.30423, -27.64971, -70.07081, -27.44275] y tiempo ('2017-06-17', '2017-06-30') ha sido guardada
Imagen de coordenada [-70.30423, -27.64971, -70.07081, -27.44275] y tiempo ('2017-06-